<a href="https://colab.research.google.com/github/riddhi-mehta-1721/NLP_Project/blob/master/Book_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP PROJECT

## RIDDHI MEHTA - J030
## HUSAIN GHADIALI - J056

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

import re

import seaborn as sns
import matplotlib.pyplot as plt

import pydotplus
import pydot

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from tensorflow.python.keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout, Conv1D, MaxPooling1D
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model
# from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
# from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model  
from keras.utils.vis_utils import model_to_dot

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

import random

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Reading Data: "positive_sentences" & "negative_sentences"

In [0]:
positive_sentences = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/positive_sentences.csv")
negative_sentences = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/negative_sentences.csv")

In [0]:
positive_sentences.drop(columns = "Unnamed: 0", inplace = True)
negative_sentences.drop(columns = "Unnamed: 0", inplace = True)

# Randomly choosing a few sentences from each dataframe which comprises of "test" data

In [0]:
pos_index = list(positive_sentences.sample(n = 10).index)

In [30]:
print(pos_index)

[45214, 14496, 27438, 63986, 36500, 38897, 14826, 33974, 48859, 15396]


In [0]:
neg_index = list(negative_sentences[negative_sentences.obscene == 1].sample(n = 20).index) + list(negative_sentences[negative_sentences.threat == 1].sample(n = 20).index) + list(negative_sentences[negative_sentences.identity_hate == 1].sample(n = 20).index)

In [0]:
neg_index = list(set(neg_index))

In [27]:
len(neg_index)

60

In [28]:
print(neg_index) 

[896, 8577, 5377, 7299, 2564, 773, 7174, 4488, 11, 3087, 5651, 1815, 3095, 6295, 2713, 4897, 8233, 4905, 7978, 8365, 4782, 1200, 3121, 4658, 3388, 1854, 4799, 6464, 5057, 4415, 6595, 7876, 3269, 7365, 6346, 5452, 3283, 2901, 4950, 5592, 5336, 1371, 4444, 7774, 4958, 2912, 4705, 8291, 1384, 7912, 5995, 5485, 3950, 6511, 3711, 6001, 1011, 3956, 1784, 5375]


In [31]:
positive_sentences.shape, negative_sentences.shape

((74136, 9), (8688, 9))

In [32]:
test = pd.concat([positive_sentences.iloc[pos_index,:], negative_sentences.iloc[neg_index,:]], axis = 0)
test.shape

(70, 9)

In [0]:
positive_sentences.drop(pos_index, inplace = True)
negative_sentences.drop(neg_index, inplace = True)

In [34]:
positive_sentences.shape, negative_sentences.shape

((74126, 9), (8628, 9))

In [35]:
test.reset_index(inplace = True)
test.drop(columns = "index", inplace = True)
test.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,positive,negative
0,among ones edited article mostly agree points ...,0,0,0,0,0,0,1,0
1,stop deleting pages stop deleting pages made p...,0,0,0,0,0,0,1,0
2,title edit wondering edit title article made i...,0,0,0,0,0,0,1,0
3,fyi edit reverted addition dubious unsourced c...,0,0,0,0,0,0,1,0
4,please vandalize pages edit prout school conti...,0,0,0,0,0,0,1,0


In [0]:
test.to_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/test.csv")

# Concatenating the modified "negative_sentences" and "positive_sentences" to form a final TRAIN data

In [0]:
train = pd.concat([positive_sentences, negative_sentences], axis = 0)

In [38]:
train.shape

(82754, 9)

In [39]:
train.reset_index(inplace = True)
train.drop(columns = "index", inplace = True)
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,positive,negative
0,explanation edits made username hardcore metal...,0,0,0,0,0,0,1,0
1,aww matches background colour seemingly stuck ...,0,0,0,0,0,0,1,0
2,hey man really trying edit war guy constantly ...,0,0,0,0,0,0,1,0
3,sorry word nonsense offensive anyway intending...,0,0,0,0,0,0,1,0
4,oh girl started arguments stuck nose belong be...,0,0,0,0,0,0,1,0


In [0]:
train.to_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/train.csv")

# ------------------------- THE END -------------------------